In [326]:
## Get list of unique words used in FR1/catFR1 tasks (so can get their semantic values using word2vec)

In [2]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

exp = 'catFR1'
poss_subs = df[df.experiment == exp]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# sub lists I've been using for sample code

# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2
# subs = ['R1112M','R1163T'] #'R1112M' # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
subs = ['R1001P']

bad_subs = ['R1433E','R1051J'] # no electrode cats

sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[1:2]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
1,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,1,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...


In [3]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3)) & # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
                    ((df.subject!='R1311T') | (df.session!=0)) & ## these next 3 eegoffset -1 for many recalls so messes things up for clustering analysis ##
                    ((df.subject!='R1113T') | (df.session!=0)) &
                    ((df.subject!='R1137E') | (df.session!=0)) 
                   ] 
if exp == 'catFR1':
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([0,1,2,3]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
exp_df.shape
# exp_df = exp_df[257:] # for catFR1 this is R1385E-onwwards
# exp_df = exp_df[472:] # for FR1 this is R1385E-onwwards
# exp_df[404:413]
exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(474, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
24,NaN,protocols/r1/subjects/R1004D/experiments/catFR...,protocols/r1/subjects/R1004D/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1004D/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1004D/localizations/0/m...,NaN,0,R1004D,R1004D,NaN,protocols/r1/subjects/R1004D/experiments/catFR...
45,NaN,protocols/r1/subjects/R1013E/experiments/catFR...,protocols/r1/subjects/R1013E/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1013E/experiments/catFR...,1,NaN,0,protocols/r1/subjects/R1013E/localizations/0/m...,NaN,0,R1013E,R1013E_1,NaN,protocols/r1/subjects/R1013E/experiments/catFR...
58,NaN,protocols/r1/subjects/R1015J/experiments/catFR...,protocols/r1/subjects/R1015J/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1015J/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1015J/localizations/0/m...,NaN,0,R1015J,R1015J,NaN,protocols/r1/subjects/R1015J/experiments/catFR...
64,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,0,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,0,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
65,NaN,protocols/r1/subjects/R1016M/experiments/catFR...,protocols/r1/subjects/R1016M/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1016M/experiments/catFR...,0,NaN,1,protocols/r1/subjects/R1016M/localizations/0/m...,NaN,1,R1016M,R1016M,NaN,protocols/r1/subjects/R1016M/experiments/catFR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,NaN,protocols/r1/subjects/R1515T/experiments/catFR...,protocols/r1/subjects/R1515T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1515T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1515T/localizations/0/m...,NaN,2,R1515T,R1515T,3.4,protocols/r1/subjects/R1515T/experiments/catFR...
2331,NaN,protocols/r1/subjects/R1515T/experiments/catFR...,protocols/r1/subjects/R1515T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1515T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1515T/localizations/0/m...,NaN,3,R1515T,R1515T,3.4,protocols/r1/subjects/R1515T/experiments/catFR...
2332,NaN,protocols/r1/subjects/R1518T/experiments/catFR...,protocols/r1/subjects/R1518T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1518T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1518T/localizations/0/m...,NaN,0,R1518T,R1518T,3.4,protocols/r1/subjects/R1518T/experiments/catFR...
2333,NaN,protocols/r1/subjects/R1518T/experiments/catFR...,protocols/r1/subjects/R1518T/localizations/0/m...,catFR1,build,0,protocols/r1/subjects/R1518T/experiments/catFR...,0,NaN,NaN,protocols/r1/subjects/R1518T/localizations/0/m...,NaN,1,R1518T,R1518T,3.4,protocols/r1/subjects/R1518T/experiments/catFR...


## LOCAL run (original program)

In [7]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload

### PARAMS ###

save_values = 0

selected_period = 'surrounding_recall' # surrounding_recall # whole_retrieval # encoding 
# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0 # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels 

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal
min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.0 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 # what I initially set it at and it seems to work well

filter_type = 'hamming' # 'butter' # Vaz used Butterworth from 80-120 Hz; Norman used Hamming from 70-180 Hz
# •Butterworth from Vaz et al: 2nd order from 80-120 ms, hilbert, select events >2 STD above mean of filtered traces.
#      Events >=25 ms long w/ max amp >3 SD were retained as ripples. Join adjacent ripples separated by <15 ms 
# •Hilbert from Norman et al: "70-180 Hz zero-lag linear-phase Hamming FIR filter w/ transition bandwidth of 5 Hz"
#      Then Hilbert, clip extreme to 4 SD, square this clipped, smooth w/ Kaiser FIR low-pass filter with 40 Hz cutoff,
#      mean and SD computed across entire experimental duration to define the threshold for event detection
#      Events from original (squared but unclipped) signal >4 SD above baseline were selected as candidate SWR events. 
#      Duration expanded until ripple power <2 SD. Events <20 ms or >200 ms excluded. Adjacent events <30 ms separation (peak-to-peak) merged.
# •Using IED detection from Vaz for 250 Hz highpass events (really 250-499). Norman uses 20-60 Hz events which is...odd.
#       See detectRipples code in module for this algorithm


# recall params
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.

### END PARAMS ###

total_word_list = []
spanish_patient_mask = np.zeros(len(exp_df))

for ct,row in enumerate(exp_df.itertuples()): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (from this list...no instrusions). 

        word_evs = evs[evs['type']=='WORD'] # get words 
        
        session_words = word_evs.item_name.unique()
        if len(findAinB(['BARRO','CABALLO','COSTILLA','DULCE','FICHA','ESTUFA','GALLINA','FRIJOL',
                        'GUARDIA','HILO','JEFE','LABIO','MAIZ','NIEVE','ORO','PALMA','PECHO','PLATO',
                        'RANA','SAPO','SERPIENTE','TALLO','UVA','VIGA','ZAPATO'],session_words))>0:
#         if len(findAinB(['AVIÓN','CAMARÓN','CORAZÓN','UÑA','HALCÓN','PELÍCULA','PEZUÑA','TELÉFONO',
#                         'MUÑECO','CÉSPED','JABÓN'],session_words))>0:
            spanish_patient_mask[ct] = 1
        else:
            total_word_list.extend(session_words) # append to master list
        
#          
    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue

total_word_list = np.unique(total_word_list)

In [10]:
test = np.unique(total_word_list)
len(test)
len(spanish_patient_mask)
sum(spanish_patient_mask)
np.where(spanish_patient_mask)

331

474

8.0

(array([ 15,  41, 351, 352, 449, 450, 451, 452]),)

In [6]:
len(total_word_list)
total_word_list
with open('listfile.txt', 'w') as filehandle:
    for listitem in total_word_list:
        filehandle.write('%s\n' % listitem)

331

array(['ALLEY', 'ANT', 'APPLE', 'ARM', 'ATTIC', 'AXE', 'BALL', 'BANANA',
       'BANJO', 'BARBIE', 'BASEBALL', 'BASKETBALL', 'BASS', 'BATHROOM',
       'BEACH', 'BEAM', 'BEAR', 'BED', 'BEE', 'BEER', 'BEETLE', 'BIKE',
       'BINDER', 'BIRCH', 'BLENDER', 'BLOCKS', 'BLOUSE', 'BLUEBERRY',
       'BLUEBIRD', 'BOAT', 'BOOKCASE', 'BROCCOLI', 'BROWNIES', 'BULL',
       'BUS', 'BUTTERFLY', 'CABBAGE', 'CABINET', 'CAKE', 'CALCULATOR',
       'CALF', 'CAMEL', 'CAR', 'CARDINAL', 'CARNATION', 'CARROT',
       'CAULIFLOWER', 'CEDAR', 'CEILING', 'CELERY', 'CELLO', 'CHAIR',
       'CHEESECAKE', 'CHEETAH', 'CHERRY', 'CHEST', 'CHICKEN',
       'CHIMPANZEE', 'CIDER', 'CLOUD', 'COAT', 'COBBLER', 'COFFEE',
       'COLA', 'COLANDER', 'COMPUTER', 'COOKIE', 'CORKSCREW', 'CORN',
       'COTTON', 'COUCH', 'COW', 'CRAB', 'CREEK', 'CRICKET', 'CROW',
       'CROWBAR', 'CUCUMBER', 'CUPCAKE', 'CYPRESS', 'DAFFODIL', 'DAISY',
       'DAY', 'DENIM', 'DESK', 'DISHWASHER', 'DOLL', 'DOLLHOUSE',
       'DOLPHIN', 'DOOR', '

6

4

6

...

5

6

5

7

6

In [22]:
# [25,  74,  75,  76, 103, 104, 105, 161, 162, 163, 394, 395, 547] are spanish patients
# [394, 395] are spanish with accents
